# BERTopic model

In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
# from cuml.preprocessing import normalize

from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 
import string
import re
import nltk
from bs4 import BeautifulSoup   
import contractions,unicodedata


nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import PorterStemmer                     # Stemmer
from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p_uli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data= pd.read_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\split_threads.csv')

In [3]:
data.head()

,Unnamed: 0,email_id,subject,email
0,0,allen-p\all_documents\1#1,"December 14, 2000 - Bear Stearns' predictions ...",In today's Daily Update you'll find free repor...
1,1,allen-p\all_documents\10#1,Bloomberg Power Lines Report,Here is today's copy of Bloomberg Power Lines....
2,2,allen-p\all_documents\100#1,Consolidated positions: Issues & To Do list,From our initial set of meetings with the trad...
3,3,allen-p\all_documents\100#2,Consolidated positions: Issues & To Do list,"Phillip,\n Below is the issues & to do list as..."
4,4,allen-p\all_documents\100#3,Consolidated positions: Issues & To Do list,


In [4]:
data.shape

(472970, 4)

## Data preprocessing

In [5]:
# removing all empty emails
data['email'] = data['email'].str.strip()
data['subject'] = data['subject'].str.strip()

In [6]:
# replace empty values for NA
data['email'].replace('', np.nan, inplace=True)

In [7]:
# drop NAs
data.dropna(subset=['email'], inplace=True)

In [8]:
data.shape

(443173, 4)

In [42]:
# utl removal
def remove_url(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return text
data['clean_text'] = data['email'].apply(lambda x: remove_url(x))

In [46]:
# html tag removal
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['clean_text'] = data['clean_text'].apply(lambda x: strip_html(x))

In [47]:
# contractions removal
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

data['clean_text'] = data['clean_text'].apply(lambda x: replace_contractions(x))
data.head()

,Unnamed: 0,email_id,subject,email,clean_text
0,0,allen-p\all_documents\1#1,"December 14, 2000 - Bear Stearns' predictions ...",In today's Daily Update you'll find free repor...,In today's Daily Update you will find free rep...
1,1,allen-p\all_documents\10#1,Bloomberg Power Lines Report,Here is today's copy of Bloomberg Power Lines....,Here is today's copy of Bloomberg Power Lines....
2,2,allen-p\all_documents\100#1,Consolidated positions: Issues & To Do list,From our initial set of meetings with the trad...,From our initial set of meetings with the trad...
3,3,allen-p\all_documents\100#2,Consolidated positions: Issues & To Do list,"Phillip,\n Below is the issues & to do list as...","Phillip,\n Below is the issues & to do list as..."
5,5,allen-p\all_documents\101#1,Consolidated positions: Issues & To Do list,From our initial set of meetings with the trad...,From our initial set of meetings with the trad...


In [48]:
# Removing numbers
def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

data['clean_text'] = data['clean_text'].apply(lambda x: remove_numbers(x))

In [49]:
# tokenization
data['clean_text'] = data.apply(lambda row: nltk.word_tokenize(row['clean_text']), axis=1) 

In [50]:
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

def lemmatize_list(words):
    new_words = []
    for word in words:
        new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize_list(words)
    return ' '.join(words)

data['clean_text'] = data.apply(lambda row: normalize(row['clean_text']), axis=1)
data.head()

,Unnamed: 0,email_id,subject,email,clean_text
0,0,allen-p\all_documents\1#1,"December 14, 2000 - Bear Stearns' predictions ...",In today's Daily Update you'll find free repor...,today daily update find free report america on...
1,1,allen-p\all_documents\10#1,Bloomberg Power Lines Report,Here is today's copy of Bloomberg Power Lines....,today copy bloomberg power line adobe acrobat ...
2,2,allen-p\all_documents\100#1,Consolidated positions: Issues & To Do list,From our initial set of meetings with the trad...,initial set meet traders regard consolidate po...
3,3,allen-p\all_documents\100#2,Consolidated positions: Issues & To Do list,"Phillip,\n Below is the issues & to do list as...",phillip issue list go forward document require...
5,5,allen-p\all_documents\101#1,Consolidated positions: Issues & To Do list,From our initial set of meetings with the trad...,initial set meet traders regard consolidate po...


In [51]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [135]:
data['clean_text']=data['clean_text'].apply(lambda x: x.replace('_',''))

In [139]:
# replace empty values for NA
data['clean_text'].replace('', np.nan, inplace=True)
# drop NAs
data.dropna(subset=['clean_text'], inplace=True)

In [140]:
data.to_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\split_threads_cleaned.csv',index=False)

In [2]:
data= pd.read_csv(r'C:\Users\p_uli\Desktop\Columbia University\Cursos\Fall 22\Capstone\Data\split_threads_cleaned.csv')

## Model

In [3]:
data.shape

(443173, 4)

In [17]:
data2= data.sample(n=20000)
if type(data2['clean_text']) is list:
    text = data2['clean_text']
else:
    text = data2['clean_text'].tolist()

text=[str(x) for x in text]

In [18]:
# Create instances of GPU-accelerated UMAP and HDBSCAN
# umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
# hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
#sentence_model = normalize(sentence_model)

In [19]:
# we add this to remove stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english', calculate_probabilities=True,
    embedding_model=sentence_model,
    verbose=True,
#     umap_model=umap_model,
#     hdbscan_model=hdbscan_model
)


In [20]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [21]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



In [22]:
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


In [24]:
%%time
with tf.device('/GPU:0'):
    topics, probs = model.fit_transform(text)


Batches:   0%|          | 0/625 [00:00<?, ?it/s]

2022-10-17 10:57:39,562 - BERTopic - Transformed documents to Embeddings
2022-10-17 10:57:59,125 - BERTopic - Reduced dimensionality
2022-10-17 10:58:55,495 - BERTopic - Clustered reduced embeddings


CPU times: total: 30min 16s
Wall time: 5min 25s


Execution times:
* 5k examples: around 1 minute
* 20k examples: around 5 minutes 13 s
* 100k examples: around (started 5:09) 21 minutes the first two steps. Did not finish after 3 hours

## Results

In [25]:
freq = model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,-1,10440,-1_thank_know_need_email
1,0,533,0_weekend_night_good_hope
2,1,498,1_meet_pm_attend_conference
3,2,270,2_deal_kate_prebon_amerex
4,3,185,3_fyi fyi_fyi_fyr fyi_fyi fyr
5,4,177,4_enron north_enron_north america_america corp
6,5,174,5_gas_thank_ena_gas market
7,6,128,6_jeff_best jeff_best_jeff jeff
8,7,124,7_agreement_draft_letter_attach
9,8,123,8_nan_nan nan_nan jin_jin nan


## Visualization

In [1]:
model.visualize_topics()

In [2]:
model.visualize_hierarchy(hierarchical_topics= topic_model.hierarchical_topics(text))

In [3]:
model.visualize_barchart()

In [4]:
from umap import UMAP
# Prepare embeddings

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(text, show_progress_bar=False)

# Train BERTopic
topic_model = BERTopic().fit(text, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(text, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(text, reduced_embeddings=reduced_embeddings)
